<a href="https://colab.research.google.com/github/DavidEnriqueOrtiz/Proyecto-1---Algoritmo-PageRank/blob/main/Proyecto_1_Algoritmo_PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

class GrafoDirigido:
    """Clase para representar un grafo dirigido utilizando matrices de adyacencia"""

    def __init__(self, nodos, arcos):
        #lista para almacenar los nodos del grafo
        self.lista_nodos = nodos
        #lista para almacenar los arcos del grafo, cada arco es una tupla (origen, destino)
        self.lista_arcos = arcos

        #matriz de adyacencia para representar conexiones entre nodos
        self.matriz_adyacencia = np.zeros((len(nodos), len(nodos)))
        for origen, destino in arcos:
            #asignar 1 en la matriz de adyacencia para indicar un arco
            self.matriz_adyacencia[origen, destino] = 1

    def __str__(self):
        #devuelve la matriz de adyacencia como una cadena legible
        return str(self.matriz_adyacencia)

    def agregar_arco(self, arco):
        """Agrega un arco a la lista de arcos y actualiza la matriz"""
        origen, destino = arco  #el arco es una tupla con nodos origen y destino
        self.lista_arcos.append(arco)  #agregar el arco a la lista de arcos
        self.matriz_adyacencia[origen, destino] = 1  #marcar el arco en la matriz

    def quitar_arco(self, arco):
        """Elimina un arco de la lista de arcos y actualiza la matriz"""
        origen, destino = arco  #el arco es una tupla con nodos origen y destino
        if arco in self.lista_arcos:  #verificar si el arco esta en la lista
            self.lista_arcos.remove(arco)  #quitar el arco de la lista
        self.matriz_adyacencia[origen, destino] = 0  #eliminar el arco de la matriz

    def contar_nodos(self):
        """Devuelve el numero de nodos"""
        #contar la longitud de la lista de nodos
        return len(self.lista_nodos)

    def contar_arcos(self):
        """Devuelve el numero total de arcos"""
        #contar la longitud de la lista de arcos
        return len(self.lista_arcos)

class RedDirigida(GrafoDirigido):
    """Clase que extiende GrafoDirigido para incluir propiedades de una red dirigida"""

    def __init__(self, nodos, arcos):
        super().__init__(nodos, arcos)
        #funcion de pesos f, inicializada como uniforme
        self.funcion_pesos = self.calcular_pesos_iniciales()

    def calcular_pesos_iniciales(self):
        """Inicializa los pesos de los arcos como uniformes"""
        pesos = np.zeros_like(self.matriz_adyacencia)
        for i in range(len(self.matriz_adyacencia)):
            suma_fila = self.matriz_adyacencia[i, :].sum()
            if suma_fila > 0:
                pesos[i, :] = self.matriz_adyacencia[i, :] / suma_fila
        return pesos

    def normalizar_matriz(self):
        """Normaliza las filas de la matriz de adyacencia para obtener probabilidades"""
        for i in range(len(self.matriz_adyacencia)):
            suma_fila = self.matriz_adyacencia[i, :].sum()  #suma de elementos en la fila i
            if suma_fila > 0:
                self.matriz_adyacencia[i, :] /= suma_fila  #dividir cada elemento entre la suma
        return self.matriz_adyacencia

    def calcular_pagerank(self, tolerancia=1e-6, max_iter=100):
        """Calcula el vector PageRank basico"""
        num_nodos = len(self.lista_nodos)  #numero total de nodos en la red
        pagerank = np.ones(num_nodos) / num_nodos  #inicializar PageRank con valores uniformes

        for _ in range(max_iter):  #limitar iteraciones para evitar bucles infinitos
            #calcular nuevo PageRank multiplicando la matriz transpuesta por el vector actual
            nuevo_pagerank = np.dot(self.funcion_pesos.T, pagerank)

            #verificar si la diferencia es menor que la tolerancia
            if np.linalg.norm(pagerank - nuevo_pagerank, ord=1) < tolerancia:
                break

            pagerank = nuevo_pagerank  #actualizar valores de PageRank

        return pagerank

    def pagerank_con_teleportacion(self, prob_teleportacion, tolerancia=1e-6, max_iter=100):
        """Calcula el vector PageRank con teleportacion"""
        num_nodos = len(self.lista_nodos)  #numero total de nodos en la red
        matriz_uniforme = np.ones((num_nodos, num_nodos)) / num_nodos  #matriz con valores uniformes

        #combinar la matriz de adyacencia con la matriz uniforme
        matriz_combinada = (prob_teleportacion * self.funcion_pesos +
                            (1 - prob_teleportacion) * matriz_uniforme)

        return self.calcular_pagerank_modificado(matriz_combinada, tolerancia, max_iter)

    def calcular_pagerank_modificado(self, matriz_modificada, tolerancia=1e-6, max_iter=100):
        """Calcula PageRank utilizando una matriz modificada"""
        num_nodos = len(self.lista_nodos)  #numero total de nodos en la red
        pagerank = np.ones(num_nodos) / num_nodos  #inicializar PageRank con valores uniformes

        for _ in range(max_iter):  #limitar iteraciones para evitar bucles infinitos
            #calcular nuevo PageRank multiplicando la matriz transpuesta por el vector actual
            nuevo_pagerank = np.dot(matriz_modificada.T, pagerank)

            #verificar si la diferencia es menor que la tolerancia
            if np.linalg.norm(pagerank - nuevo_pagerank, ord=1) < tolerancia:
                break

            pagerank = nuevo_pagerank  #actualizar valores de PageRank

        return pagerank

def construir_red_desde_excel(ruta_archivo):
    """Construye una red dirigida desde un archivo Excel"""
    datos = pd.read_excel(ruta_archivo)  #leer archivo Excel con datos
    nodos = list(range(len(datos)))  #crear lista de nodos, uno por cada fila de datos
    arcos = []  #inicializar lista para almacenar arcos

    for indice, citas in enumerate(datos["Cited by"]):  #iterar por cada fila y sus citas
        for citado in map(int, citas.split(",")):  #dividir las citas en una lista de enteros
            arcos.append((citado - 1, indice))  #agregar cada arco como una tupla (origen, destino)

    return RedDirigida(nodos, arcos), datos  #devolver la red y los datos del Excel

def main():
    """Funcion principal para realizar los calculos"""
    ruta_archivo_excel = '/content/Web.xlsx'  #ruta al archivo Excel con los datos
    red, datos = construir_red_desde_excel(ruta_archivo_excel)  #construir red a partir del archivo

    print("\n--- Pregunta 1: PageRank basico ---")
    pagerank_basico = red.calcular_pagerank()  #calcular PageRank basico
    print("Vector PageRank basico:", pagerank_basico)  #imprimir el vector PageRank
    print("Pagina mas importante (indice):", np.argmax(pagerank_basico))  #mostrar el indice de la pagina mas importante

    print("\n--- Pregunta 2: PageRank modificado ---")
    #crear una lista que indica si cada pagina termina en .ru (1 para si, 0 para no)
    filtro_ru = datos["Website"].str.endswith(".ru").astype(int).tolist()
    #lista con los nombres de las paginas que terminan en .ru
    paginas_ru = datos.loc[datos["Website"].str.endswith(".ru"), "Website"].tolist()

    if sum(filtro_ru) == 0:  #verificar si hay paginas con terminacion .ru
        print("No hay paginas con terminacion .ru")
    else:
        print("Paginas con terminacion .ru:", paginas_ru)  #imprimir nombres de las paginas
        matriz_modificada = np.multiply(red.funcion_pesos, filtro_ru)  #aplicar el filtro a la matriz
        pagerank_modificado = red.calcular_pagerank_modificado(matriz_modificada)  #calcular PageRank modificado
        print("Vector PageRank modificado:", pagerank_modificado)  #imprimir el resultado

    print("\n--- Pregunta 3: PageRank con teleportacion ---")
    for prob in [0.5, 0.85, 1]:  #iterar con diferentes probabilidades de teleportacion
        pagerank_teleportacion = red.pagerank_con_teleportacion(prob)  #calcular PageRank con teleportacion
        print(f"\nVector PageRank con probabilidad de teleportacion={prob}:", pagerank_teleportacion)  #imprimir resultado

if __name__ == "__main__":
    main()



--- Pregunta 1: PageRank basico ---
Vector PageRank basico: [1.84989458e-05 2.56713174e-05 1.59775013e-05 3.43293657e-05
 1.34885265e-05 2.07022315e-05 3.45613804e-05 2.10368844e-05
 2.77915464e-05 2.42345186e-05 1.27266794e-05 1.32196211e-05
 1.90251410e-05 1.91134333e-05 2.45550599e-05 3.60576923e-02
 3.60576923e-02 3.60576923e-02 3.60576923e-02 4.80769231e-02
 3.60576923e-02 3.60576923e-02 3.60576923e-02 4.80769231e-02
 3.60576923e-02 6.15059683e-01]
Pagina mas importante (indice): 25

--- Pregunta 2: PageRank modificado ---
Paginas con terminacion .ru: ['yandex.ru', 'lenta.ru', 'gazeta.ru', 'ria.ru', 'rbk.ru', 'tass.ru', 'iz.ru', 'kommersant.ru', 'vedomosti.ru', 'regnum.ru']
Vector PageRank modificado: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.0360576  0.03605779 0.03605767
 0.03605774 0.04807692 0.03605774 0.03605757 0.03605762 0.048077
 0.03605773 0.        ]

--- Preg

###Low-Level Design (Diseño a bajo nivel)

**Clases y metodos principales:**
1. **Clase `GrafoDirigido`:**
   - Representa un grafo dirigido mediante una matriz de adyacencia.
   - Proporciona metodos para agregar, quitar arcos y obtener informacion basica sobre el grafo (numero de nodos y arcos).

   ```python
   class GrafoDirigido:
       def __init__(self, nodos, arcos):
           self.lista_nodos = nodos
           self.lista_arcos = arcos
           self.matriz_adyacencia = np.zeros((len(nodos), len(nodos)))
           for origen, destino in arcos:
               self.matriz_adyacencia[origen, destino] = 1

       def agregar_arco(self, arco):
           origen, destino = arco
           self.lista_arcos.append(arco)
           self.matriz_adyacencia[origen, destino] = 1

       def quitar_arco(self, arco):
           origen, destino = arco
           if arco in self.lista_arcos:
               self.lista_arcos.remove(arco)
           self.matriz_adyacencia[origen, destino] = 0
   ```

2. **Clase `RedDirigida` (hereda de `GrafoDirigido`):**
   - Extiende las funcionalidades del grafo para incluir calculos especificos de una red dirigida.
   - Define una funcion de pesos inicial uniforme (`calcular_pesos_iniciales`) basada en las conexiones entre nodos.

   ```python
   class RedDirigida(GrafoDirigido):
       def __init__(self, nodos, arcos):
           super().__init__(nodos, arcos)
           self.funcion_pesos = self.calcular_pesos_iniciales()

       def calcular_pesos_iniciales(self):
           pesos = np.zeros_like(self.matriz_adyacencia)
           for i in range(len(self.matriz_adyacencia)):
               suma_fila = self.matriz_adyacencia[i, :].sum()
               if suma_fila > 0:
                   pesos[i, :] = self.matriz_adyacencia[i, :] / suma_fila
           return pesos
   ```

3. **Metodos de `RedDirigida`:**
   - **`calcular_pagerank`:** Calcula el vector de importancia relativa de los nodos.
   - **`pagerank_con_teleportacion`:** Incluye teleportacion para evitar problemas de ciclos absorbentes.
   - **`calcular_pagerank_modificado`:** Metodo general para trabajar con cualquier matriz de transicion modificada.

   ```python
       def calcular_pagerank(self, tolerancia=1e-6, max_iter=100):
           num_nodos = len(self.lista_nodos)
           pagerank = np.ones(num_nodos) / num_nodos

           for _ in range(max_iter):
               nuevo_pagerank = np.dot(self.funcion_pesos.T, pagerank)
               if np.linalg.norm(pagerank - nuevo_pagerank, ord=1) < tolerancia:
                   break
               pagerank = nuevo_pagerank

           return pagerank

       def pagerank_con_teleportacion(self, prob_teleportacion, tolerancia=1e-6, max_iter=100):
           num_nodos = len(self.lista_nodos)
           matriz_uniforme = np.ones((num_nodos, num_nodos)) / num_nodos
           matriz_combinada = (prob_teleportacion * self.funcion_pesos +
                               (1 - prob_teleportacion) * matriz_uniforme)
           return self.calcular_pagerank_modificado(matriz_combinada, tolerancia, max_iter)
   ```

4. **Funcion `construir_red_desde_excel`:**
   - Lee datos de un archivo Excel, genera nodos y arcos, y construye una red (`RedDirigida`).

   ```python
   def construir_red_desde_excel(ruta_archivo):
       datos = pd.read_excel(ruta_archivo)
       nodos = list(range(len(datos)))
       arcos = []

       for indice, citas in enumerate(datos["Cited by"]):
           for citado in map(int, citas.split(",")):
               arcos.append((citado - 1, indice))

       return RedDirigida(nodos, arcos), datos
   ```

---

###Explicacion del Codigo

1. **Clase `GrafoDirigido`:**
   - Inicializa una lista de nodos (`self.lista_nodos`) y arcos (`self.lista_arcos`).
   - La matriz de adyacencia (`self.matriz_adyacencia`) indica si hay una conexion entre dos nodos con un valor de 1.

2. **Clase `RedDirigida`:**
   - La funcion de pesos (`self.funcion_pesos`) se normaliza para representar las probabilidades de transicion desde cada nodo.
   - **Normalizacion:** Si un nodo tiene multiples salidas, sus valores se dividen entre la suma total de sus conexiones.
   - **PageRank basico:** Calcula la importancia relativa de cada nodo iterando hasta que el vector converja.
   - **Teleportacion:** Introduce una probabilidad fija para saltar aleatoriamente a cualquier nodo, resolviendo problemas de ciclos absorbentes y nodos sin salidas.

3. **Funcion `construir_red_desde_excel`:**
   - Convierte datos de un archivo Excel en una red.
   - Genera una lista de nodos numerados y una lista de arcos con los valores de las citas proporcionadas en el Excel.

4. **`main`:**
   - Responde a tres preguntas usando la red:
     - PageRank basico.
     - PageRank modificado con un filtro para paginas `.ru`.
     - PageRank con teleportacion para diferentes probabilidades (\(d\)).

---

###Explicacion de Resultados

1. **Pregunta 1: PageRank basico**
   - **Resultado:**  
     ```python
     Vector PageRank basico: [1.84989458e-05 ... 6.15059683e-01]
     Pagina mas importante (indice): 25
     ```
   - **Analisis:**
     - El nodo 25 tiene el mayor valor de PageRank (0.615).
     - Esto indica que es el nodo mas relevante en la red, probablemente debido a una alta cantidad de conexiones entrantes desde otros nodos.

2. **Pregunta 2: PageRank modificado**
   - **Resultado:**  
     ```python
     Vector PageRank modificado: [0. ... 0.03605773 0. ...]
     ```
   - **Analisis:**
     - Se aplica un filtro que favorece paginas `.ru`.
     - Los valores de PageRank se concentran exclusivamente en estos nodos, lo que muestra que los demas nodos tienen cero probabilidad de ser alcanzados bajo esta modificacion.

3. **Pregunta 3: PageRank con teleportacion**
   - **Resultados con \(d = 0.5, 0.85, 1\):**
     ```python
     d=0.5: [0.03276549 ... 0.07399901]
     d=0.85: [0.02375506 ... 0.20000987]
     d=1: [1.84989458e-05 ... 6.15059683e-01]
     ```
   - **Analisis:**
     - Con \(d = 0.5\), el PageRank esta mas distribuido entre los nodos debido a una mayor probabilidad de teleportacion.
     - Con \(d = 0.85\), el nodo 25 sigue destacandose, pero con menor dominancia.
     - Con \(d = 1\), volvemos al calculo del PageRank basico, lo que refuerza la posicion dominante del nodo 25.


